In [1]:
from pop_llm import PopFunction
import openai

openai.api_key_path = "openai_api_key.txt"

# Demo POP Functions

## Add two numbers

In [13]:
add = PopFunction(
    input_keys=["a", "b"],
    output_keys=["sum"],
    description="Add two numbers.",
    name = "add",
    input_assert=lambda a, b: type(a) == int and type(b) == int,
)
answer = int(add(23, 465)["sum"])
answer

488

## Generate TL;DR summary

In [20]:
tldr = PopFunction(
    input_keys=["text"],
    output_keys=["summary"],
    description="Generate a TL;DR summary of a text.",
    name = "tldr",
    input_assert=lambda text: type(text) == str,
)

# This article is from https://en.wikipedia.org/wiki/Linux
article = """
Linux (/ˈliːnʊks/ (listen) LEE-nuuks or /ˈlɪnʊks/ LIN-uuks)[11] is a family of open-source Unix-like operating systems based on the Linux kernel,[12] an operating system kernel first released on September 17, 1991, by Linus Torvalds.[13][14][15] Linux is typically packaged as a Linux distribution, which includes the kernel and supporting system software and libraries, many of which are provided by the GNU Project. Many Linux distributions use the word "Linux" in their name, but the Free Software Foundation uses the name "GNU/Linux" to emphasize the importance of GNU software, causing some controversy.[16][17]

Popular Linux distributions[18][19][20] include Debian, Fedora Linux, and Ubuntu, the latter of which itself consists of many different distributions and modifications, including Lubuntu and Xubuntu. Commercial distributions include Red Hat Enterprise Linux and SUSE Linux Enterprise. Desktop Linux distributions include a windowing system such as X11 or Wayland, and a desktop environment such as GNOME or KDE Plasma. Distributions intended for servers may omit graphics altogether, or include a solution stack such as LAMP. Because Linux is freely redistributable, anyone may create a distribution for any purpose.[21]

Linux was originally developed for personal computers based on the Intel x86 architecture, but has since been ported to more platforms than any other operating system.[22] Because of the dominance of the Linux-based Android on smartphones, Linux, including Android, has the largest installed base of all general-purpose operating systems, as of May 2022.[23][24][25] Although Linux is, as of November 2022, used by only around 2.6 percent of desktop computers,[26] the Chromebook, which runs the Linux kernel-based ChromeOS, dominates the US K–12 education market and represents nearly 20 percent of sub-$300 notebook sales in the US.[27] Linux is the leading operating system on servers (over 96.4% of the top 1 million web servers' operating systems are Linux),[28] leads other big iron systems such as mainframe computers, and is used on all of the world's 500 fastest supercomputers[d] (since November 2017, having gradually displaced all competitors).[29][30][31]

Linux also runs on embedded systems, i.e. devices whose operating system is typically built into the firmware and is highly tailored to the system. This includes routers, automation controls, smart home devices, video game consoles,[32] televisions (Samsung and LG Smart TVs),[33][34][35] automobiles (Tesla, Audi, Mercedes-Benz, Hyundai and Toyota),[36] and spacecraft (Falcon 9 rocket, Dragon crew capsule and the Perseverance rover).[37][38]

Linux is one of the most prominent examples of free and open-source software collaboration. The source code may be used, modified and distributed commercially or non-commercially by anyone under the terms of its respective licenses, such as the GNU General Public License (GPL). The Linux kernel, for example, is licensed under the GPLv2, with an exception for system calls that allows code that calls the kernel via system calls not to be licensed under the GPL.[39][40][21]

"""
summary = tldr(article)["summary"]
summary = "\n".join([summary[i:i+100] for i in range(0, len(summary), 100)])
print(summary)

Linux is a family of open-source Unix-like operating systems based on the Linux kernel, which was fi
rst released on September 17, 1991, by Linus Torvalds. Linux is typically packaged as a Linux distri
bution, which includes the kernel and supporting system software and libraries. Popular Linux distri
butions include Debian, Fedora Linux, and Ubuntu. Linux is used on personal computers, servers, and 
embedded systems, including routers, automation controls, smart home devices, video game consoles, t
elevisions, automobiles, and spacecraft. Linux is one of the most prominent examples of free and ope
n-source software collaboration.


## Generate Bash command

In [21]:
bash = PopFunction(
    input_keys=["task"],
    output_keys=["command"],
    description="Generate a bash command to do a task.",
    name = "bash",
    input_assert=lambda task: type(task) == str,
)

task = """
Write a single command that:
1. Create a directory called "pop"
2. Change directory to "pop"
3. Initialize a git repository
4. Create a file called "README.md", with the content "This is a Pop repository"
5. Add all files to git
6. Commit with the message "Initial commit"
"""
command = bash(task)["command"]
print(command)

mkdir pop && cd pop && git init && echo 'This is a Pop repository' > README.md && git add . && git commit -m 'Initial commit'


## Chat with history!

In [2]:
chatbot = PopFunction(
    input_keys=["message"],
    output_keys=["response"],
    description="Chat with your user.",
    name = "chatbot",
    input_assert=lambda message: type(message) == str,
    temperature=1
)

print(chatbot("Hello, how are you?")["response"])
print(chatbot("Remember my name is John.")["response"])
print(chatbot("What is my name?")["response"])
chatbot.reset_history()
print(chatbot("What is my name?")["response"])

I'm an AI language model, I don't have emotions in the way humans do, but I'm here to serve you. How may I assist you?
Sure John, your name is remembered. How may I assist you further?
Your name is John. Is there anything else I can help you with?
I'm sorry, I cannot determine your name.


## Use external tools

In [4]:
def read_file(path: str) -> str:
    """
    Read the content of a file.

    Args:
        path (str): path to the file

    Returns:
        str: file content
    """
    with open(path, "r") as f:
        return f.read()

# Create a function similar to the tldr function above, but with tools.
tldr_with_tools = PopFunction(
    input_keys=["text"],
    output_keys=["summary"],
    description="Generate a TL;DR summary of a text.",
    name = "tldr",
    input_assert=lambda text: type(text) == str,
    tools=[read_file]
)

print(tldr_with_tools.system_prompt)

print(tldr_with_tools("Summarize README.md")["summary"])


You are a function that strictly takes inputs from the user and return a JSON formatted output.
Think about this step by step:
1. You will be given a task description.
2. You will be given a list of input keys.
3. You will be given a list of output keys.
4. User will give you a JSON formatted input.
5. You will follow the task description to process the input.
6. You will return a JSON formatted output, which contains the output keys.

Rules:
1. Unless you are specifically asked to write code, you should directly process the inputs and return the outputs by yourself.
2. You can ONLY reply with a JSON formatted string.
3. You ONLY reply with the given output keys.
Your name is: tldr.
Your task is described as follows:
    Generate a TL;DR summary of a text.
Your input keys are: 
["text"].
Your output keys are: 
["summary"].


You can use tools to help you solve the task. However, YOU MUST ask the user for permission before using a tool.
After you use a tool, system will provide you the

Sometimes the stupid GPT-3.5 will try to use a non-existent tool to complete the task. It's okay because we correct it's mistakes by providing error feedback.